In [1]:
import pandas as pd
import s3fs


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
1,application_1594660132298_0003,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [ ]:
# Redshift Server Details
dsn_database = "dev"
dsn_hostname = "redshift-cluster-1.citpxgaovgkr.eu-north-1.redshift.amazonaws.com"
dsn_port = "5439"
dsn_uid = "awsuser"
dsn_database = "telcom"
jdbcUrl = 'jdbc:redshift:iam://'+ dsn_hostname+':'+dsn_port+'/'+dsn_database


In [2]:
(df = spark.read.
      format("jdbc").
      option("driver", "com.amazon.redshift.jdbc42.Driver").
      option("url", jdbcUrl).
      option("dbtable", dsn_database).
      option("user", jdbcUsername).
      load()
)

type(df)

<class 'pyspark.sql.dataframe.DataFrame'>

In [5]:
#from hops import hdfs as hdfs
#df.write.format("delta").save(hdfs.project_path() + "Resources/telcom-delta")
df.write.format("delta").save("s3a://sagemakerjim/telco-delta")

In [5]:
from pyspark.sql.types import *

# Auxiliar functions
def equivalent_type(f):
    if f == 'datetime64[ns]': return DateType()
    elif f == 'int64': return LongType()
    elif f == 'int32': return IntegerType()
    elif f == 'float64': return FloatType()
    else: return StringType()

def define_structure(string, format_type):
    try: typo = equivalent_type(format_type)
    except: typo = StringType()
    return StructField(string, typo)


# Given pandas dataframe, it will return a spark's dataframe.
def pandas_to_spark(pandas_df):
    columns = list(pandas_df.columns)
    types = list(pandas_df.dtypes)
    struct_list = []
    for column, typo in zip(columns, types): 
      struct_list.append(define_structure(column, typo))
    p_schema = StructType(struct_list)
    return sqlContext.createDataFrame(pandas_df, p_schema)

In [7]:
df.registerTempTable("telcom")

In [9]:
sqlContext.sql("describe telcom")

DataFrame[col_name: string, data_type: string, comment: string]

In [11]:
df.describe()

DataFrame[summary: string, customer_id: string, gender: string, SeniorCitizen: string, Partner: string, Dependents: string, tenure: string, PhoneService: string, MultipleLines: string, InternetService: string, OnlineSecurity: string, OnlineBackup: string, DeviceProtection: string, TechSupport: string, StreamingTV: string, StreamingMovies: string, Contract: string, PaperlessBilling: string, PaymentMethod: string, MonthlyCharges: string, TotalCharges: string, Churn: string]